In [1]:
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdb
from IPython.display import display
import ee
import os

In [2]:
import ee
import geemap
service_account = ' boba-account@boba-430314.iam.gserviceaccount.com '
credentials = ee.ServiceAccountCredentials(service_account,'C:/Users/arj26323/OneDrive - University of Georgia/Documents/GEE AUTH/boba-430314-a19be859bfca.json')

ee.Initialize(credentials)

print(geemap.__version__)

0.34.1


In [3]:
#ee.Authenticate()
#geemap.update_package()

# ee.Initialize()

Map = geemap.Map(center=[37.395746,-75.876094], zoom=10)

##Adding every plot coordinate
# allplots_fc= pd.read_csv('C:/Users/arj26323/Documents/Data/Biomass datasets/Virginia/Coordinates and biomass/va_allpoints.csv')
allplots_fc= pd.read_csv('C:/Users/arj26323/Documents/Data/Biomass datasets/Virginia/Coordinates and biomass/va_allspecies.csv')

allplots_fc = allplots_fc[allplots_fc['liveMass'].notna()]
allplots_fc = allplots_fc[allplots_fc['Latitude'].notna()]
allplots_fc = allplots_fc.drop(columns = ['deadMass', 'collectDate', 'totalMass', 'Ell', 'MSL', 'latitude', 'longitude'])

##Group df1 and average (midpoint) the replicates (a/b)
df_temp = allplots_fc.copy()

df_temp = allplots_fc.groupby(['Year','Latitude','Longitude'],as_index = False).aggregate(
    {
        'liveMass':[np.mean, np.size]
    }
)

df_temp.columns = [
    'Year','Latitude','Longitude','liveMass','N'
]


# display(allplots_fc)

fc_all = geemap.pandas_to_ee(df_temp, latitude = "Latitude", longitude = "Longitude")

Map.addLayer(fc_all, {}, "fc_all")

In [4]:
df_temp

# Map

,Year,Latitude,Longitude,liveMass,N
0,1999,37.167147,-75.940768,46.720,2
1,1999,37.167148,-75.943672,126.720,4
2,1999,37.167154,-75.941307,164.960,2
3,1999,37.167195,-75.944905,83.776,5
4,1999,37.167409,-75.940586,154.000,2
...,...,...,...,...,...
1389,2017,37.450439,-75.672455,43.440,4
1390,2017,37.450771,-75.672169,168.480,2
1391,2017,37.451099,-75.673269,158.240,4
1392,2018,37.396051,-75.876056,127.520,2


In [5]:
##Function to cloud mask from the pixel_qa band of Landsat 5/8 SR data.

def maskL8sr(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)
    # Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    # Replace the original bands with the scaled ones and apply the masks.
    return image.addBands(opticalBands, None, True) \
    .addBands(thermalBands, None, True) \
    .updateMask(qaMask) \
    .updateMask(saturationMask)

#NOTE 10/6/2022 - This has been updated for Landsat Collection 2 https://www.usgs.gov/landsat-missions/landsat-collection-2
#Scaling factors: https://www.usgs.gov/landsat-missions/landsat-collection-2-level-2-science-products

In [6]:
##TIDAL FILTERING; from Narron et al. 2022
##Utilizes L8 bands 4 and 6 for NDWI, and bands 3 and 4 (for pheno)
##Does it work for Landsat 5?

#12/08/22 - adding CALIBRATED FLATS
def addFLATSL7(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(RED-SWIR)/(RED+SWIR) - 41.2*(NIR-RED)/(NIR+6*RED-7.5*BLUE+1)))', {
            'SWIR': image.select('SR_B5'),
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3'),
            'BLUE': image.select('SR_B1')
        })
    
    return image.addBands(flats.rename('flats'))

def addFLATSL5(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(0.972*(RED-SWIR)/(RED+SWIR)-0.008) - 41.2*(0.991*(NIR-RED)/(NIR+6*RED-7.5*BLUE+1)-0.0014)))', {
            'SWIR': image.select('SR_B5'),
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3'),
            'BLUE': image.select('SR_B1')
        })
    
    return image.addBands(flats.rename('flats'))


def addFLATSL8(image):
    flats = ee.Image(0).expression(
        '1/(1+2.718281828459045**-(1.51 + 12.5*(0.841*(RED-SWIR)/(RED+SWIR) - 0.019) - 41.2*(0.771*(NIR-RED)/(NIR+6*RED-7.5*BLUE+1) + 0.011)))', {
            'SWIR': image.select('SR_B6'),
            'NIR': image.select('SR_B5'),
            'RED': image.select('SR_B4'),
            'BLUE': image.select('SR_B2')
        })
    
    return image.addBands(flats.rename('flats'))

#Updated 10/6/2022 - Band names changed for Collection 2

##MASKING FLATS
def maskFLATS(image):
    mask1 = image.select('flats').lte(0.1) #less than or equal to 0.1 - change?
    return image.updateMask(mask1)

##ADDING NDVI (for min/max variables)
def addL5ndvi(image):
    ndvi = image.expression(
        '(NIR-RED)/(RED+NIR)', {
            'NIR': image.select('SR_B4'),
            'RED': image.select('SR_B3'),
            'GREEN': image.select('SR_B2')
        })
    
    return image.addBands(ndvi.rename('ndvi'))

def addL8ndvi(image):
    ndvi = image.expression(
        '(NIR-RED)/(RED+NIR)', {
            'NIR': image.select('SR_B5'),
            'RED': image.select('SR_B4'),
            'GREEN': image.select('SR_B3')
        })
    
    return image.addBands(ndvi.rename('ndvi'))

In [7]:
##Pixel extraction functions - addDate for dateless images/collections
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('imagedate').toInt())

##For Landsat images:
def rasterExtraction(image):
    feature = image.sampleRegions(
        collection = fc_all,
        scale = 30,
        tileScale = 8 #ADDED 10/6/2022 - make sure it doesn't affect results

    )
    return feature

##FOR 10m DEM:
def demExtraction(image):
    feature = image.sampleRegions(
        collection = fc_all,
        scale = 10 
    )
    return feature

##FOR 1m DEM:
def dem1Extraction(image):
    feature = image.sampleRegions(
        collection = fc_all,
        scale = 1 
    )
    return feature

#tileScale: https://gis.stackexchange.com/questions/373250/understanding-tilescale-in-earth-engine

In [8]:
##Adding DEM
dem = ee.Image('USGS/3DEP/10m') ##This is 1/3 arc second, or 10 m.
dem1 = ee.ImageCollection('USGS/3DEP/1m')

##Set visualization parameters.
dem_params = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'],
}

Map.addLayer(dem, dem_params, '10m DEM')
Map.addLayer(dem1, dem_params, '1m DEM')

In [9]:
##Calculating mean pixel values for time periods within each year

def monthly_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(5, 10, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)

## Compute monthly averages
# monthly_sowing_Avg = monthly_Avg (ndvi_sowSeason, years, sowingMonths)

##Months and years are lists

years_ls5 = range(1999, 2012)
years_ls7 = range(2012, 2013)
years_ls8 = range(2013, 2021)
years_dm = range(2000, 2021)

months = range(5,11)
months_daymet = range(1,12)

#Note:Landsat datasets migrated to Collection 2

ls5_collect = ee.ImageCollection(
    'LANDSAT/LT05/C02/T1_L2'
).filterBounds(fc_all).map(maskL8sr).map(addFLATSL5).map(maskFLATS).map(addL5ndvi) ##MIGRATE TO COLLECTION 2 DATA

ls7_collect = ee.ImageCollection(
    'LANDSAT/LE07/C02/T1_L2'
).filterBounds(fc_all).map(maskL8sr).map(addFLATSL7).map(maskFLATS).map(addL5ndvi) ##MIGRATE TO COLLECTION 2 DATA

ls8_collect = ee.ImageCollection(
    'LANDSAT/LC08/C02/T1_L2'
).filterBounds(fc_all).map(maskL8sr).map(addFLATSL8).map(maskFLATS).map(addL8ndvi) ##NEW L8 DATASET: "LANDSAT/LC08/C02/T1_L2"

monthly_ls5 = monthly_Avg(ls5_collect, years = years_ls5)
monthly_ls7 = monthly_Avg(ls7_collect, years = years_ls7)
monthly_ls8 = monthly_Avg(ls8_collect, years = years_ls8)

monthly_ls5.size().getInfo()

13

In [40]:
def year_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(1, 12, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)


year_ls5 = year_Avg(ls5_collect, years = years_ls5)
year_ls7 = year_Avg(ls7_collect, years = years_ls7)
year_ls8 = year_Avg(ls8_collect, years = years_ls8)

yearlist_5 = year_ls5.toList(year_ls5.size())
yearlist_7 = year_ls7.toList(year_ls7.size())
yearlist_8 = year_ls8.toList(year_ls8.size())

def peak_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(8, 10, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)

peak_ls5 = peak_Avg(ls5_collect, years = years_ls5)
peak_ls7 = peak_Avg(ls7_collect, years = years_ls7)
peak_ls8 = peak_Avg(ls8_collect, years = years_ls8)

peaklist_5 = peak_ls5.toList(peak_ls5.size())
peaklist_7 = peak_ls7.toList(peak_ls7.size())
peaklist_8 = peak_ls8.toList(peak_ls8.size())

def early_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(3, 7, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)

early_ls5 = early_Avg(ls5_collect, years = years_ls5)
early_ls7 = early_Avg(ls7_collect, years = years_ls7)
early_ls8 = early_Avg(ls8_collect, years = years_ls8)

earlylist_5 = early_ls5.toList(early_ls5.size())
earlylist_7 = early_ls7.toList(early_ls7.size())
earlylist_8 = early_ls8.toList(early_ls8.size())


def MayOct_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(5, 10, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)


MayOct_ls5 = MayOct_Avg(ls5_collect, years = years_ls5)
MayOct_ls7 = MayOct_Avg(ls7_collect, years = years_ls7)
MayOct_ls8 = MayOct_Avg(ls8_collect, years = years_ls8)

MayOctlist_5 = MayOct_ls5.toList(MayOct_ls5.size())
MayOctlist_7 = MayOct_ls7.toList(MayOct_ls7.size())
MayOctlist_8 = MayOct_ls8.toList(MayOct_ls8.size())


def MayOct_Max (collection, years):
  maxim = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_maxim = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(5, 10, 'month')) \
                              .max() \
                              .set({'year': year})
      maxim.append (Monthly_maxim)
  return ee.ImageCollection.fromImages(maxim)

Max_ls5 = MayOct_Max(ls5_collect, years = years_ls5)
Max_ls7 = MayOct_Max(ls7_collect, years = years_ls7)
Max_ls8 = MayOct_Max(ls8_collect, years = years_ls8)

Maxlist_5 = Max_ls5.toList(Max_ls5.size())
Maxlist_7 = Max_ls7.toList(Max_ls7.size())
Maxlist_8 = Max_ls8.toList(Max_ls8.size())



def MayOct_Min (collection, years):
  minim = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter and set month
      Monthly_minim = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(5, 10, 'month')) \
                              .max() \
                              .set({'year': year})
      minim.append (Monthly_minim)
  return ee.ImageCollection.fromImages(minim)

Min_ls5 = MayOct_Min(ls5_collect, years = years_ls5)
Min_ls7 = MayOct_Min(ls7_collect, years = years_ls7)
Min_ls8 = MayOct_Min(ls8_collect, years = years_ls8)

Minlist_5 = Min_ls5.toList(Min_ls5.size())
Minlist_7 = Min_ls7.toList(Min_ls7.size())
Minlist_8 = Min_ls8.toList(Min_ls8.size())


In [41]:
#Daymet
def dm_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(1, 12, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)


def dm_earlyseason (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(3, 7, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)


def peakdm_Avg (collection, years):
  avg = []
  for year in years: #Originally had a for month in months subloop, with (month,month,'month') being a filter
      Monthly_avg = collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                              .filter(ee.Filter.calendarRange(8, 10, 'month')) \
                              .mean() \
                              .set({'year': year})
      avg.append (Monthly_avg)
  return ee.ImageCollection.fromImages(avg)

def rolling_average(daymet_ic):
    # Get the start and end years from the collection
    start_date = ee.Date(daymet_ic.first().get('system:time_start'))
    end_date = ee.Date(daymet_ic.sort('system:time_start', False).first().get('system:time_start'))
    start_year = start_date.get('year').getInfo()
    end_year = end_date.get('year').getInfo()
    
    rolling_avg_ic = ee.ImageCollection([])
    
    # Loop over each year in the collection's range, starting from the third year
    for year in range(start_year + 2, end_year + 1):
        # Define the 3-year range for each year
        start_date = ee.Date.fromYMD(year - 2, 1, 1)
        end_date = ee.Date.fromYMD(year, 12, 31)
        
        # Filter the collection to this 3-year range and calculate mean
        three_year_avg = daymet_ic.filterDate(start_date, end_date).mean()
        
        # Set the year as a property to keep track
        three_year_avg = three_year_avg.set('year', year)
        
        # Add to the rolling average collection
        rolling_avg_ic = rolling_avg_ic.merge(ee.ImageCollection([three_year_avg]))

    return rolling_avg_ic

##COLLECTION
daymet = ee.ImageCollection('NASA/ORNL/DAYMET_V4').filterBounds(fc_all)

#Resampling daymet collection to 30m
def resamp_dm(img):
    resampled_dm = img.reproject(
        crs = 'EPSG:4326',
        scale = 30
    ).resample('bilinear')
    return resampled_dm
resamp_daymet = daymet.map(resamp_dm) #not sure if anything is actually changed

#Over year
monthly_dm = dm_Avg(resamp_daymet, years = years_dm) #Updated to resampled daymet collection!
dm_list = monthly_dm.toList(monthly_dm.size())

#Peak biomass
peak_dm = peakdm_Avg(resamp_daymet, years = years_dm) #Updated to resampled daymet collection!
peakdm_list = peak_dm.toList(peak_dm.size())

#Previous 3 years, rolling average
rolling_av = rolling_average(resamp_daymet) 
rolling_list = rolling_av.toList(rolling_av.size())

#Early season
early_dm = dm_earlyseason(resamp_daymet, years = years_dm) #Updated to resampled daymet collection!
early_list = early_dm.toList(early_dm.size())

#Growing season
MayOct_dm = MayOct_Avg(resamp_daymet, years = years_dm) #Updated to resampled daymet collection!
dm_MayOct_list = MayOct_dm.toList(MayOct_dm.size())

#Max
Max_dm = MayOct_Max(resamp_daymet, years = years_dm) #Updated to resampled daymet collection!
dm_Max_list = Max_dm.toList(Max_dm.size())

#Min
Min_dm = MayOct_Min(resamp_daymet, years = years_dm) #Updated to resampled daymet collection!
dm_Min_list = Min_dm.toList(Min_dm.size())

In [12]:
#Elevation
dem_extract = demExtraction(dem)
dem_dict = dem_extract.getInfo()
# Extract the properties of the features into a list of dictionaries
features = [feature['properties'] for feature in dem_dict['features']]
# Convert the list of dictionaries into a pandas DataFrame
dem_vals = pd.DataFrame(features)
dem_vals

# dem_vals = geemap.ee_to_pandas(demExtraction(dem)) ##10m dataset NO MORE EE_TO_PANDAS
# dem_vals = geemap.ee_to_pandas(dem1.map(dem1Extraction).flatten()) ##ONE METER DATASET

,Latitude,Longitude,N,Year,elevation,liveMass
0,37.167147,-75.940768,2,1999,0.157865,46.720
1,37.167148,-75.943672,4,1999,0.668236,126.720
2,37.167154,-75.941307,2,1999,0.495402,164.960
3,37.167195,-75.944905,5,1999,0.908289,83.776
4,37.167409,-75.940586,2,1999,0.328943,154.000
...,...,...,...,...,...,...
1389,37.450439,-75.672455,4,2017,1.229311,43.440
1390,37.450771,-75.672169,2,2017,1.246447,168.480
1391,37.451099,-75.673269,4,2017,0.619497,158.240
1392,37.396051,-75.876056,2,2018,0.957294,127.520


In [13]:
list_5 = monthly_ls5.toList(monthly_ls5.size())
list_7 = monthly_ls7.toList(monthly_ls7.size())
list_8 = monthly_ls8.toList(monthly_ls8.size())

In [14]:
##Monthly/yearly extraction - NOTE: Takes a while to run
##SUBSET BY YEAR

In [18]:
#MayOCT?:
years_l5 = range(1999, 2012)
years_l7 = range(2012, 2013)
years_l8 = range(2013, 2019)

landsat5_list = []
for i in range(len(years_l5)):
    ls5_x = rasterExtraction(ee.Image(list_5.get(i)))
    fc_dict = ls5_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample5 = ydf[ydf['Year'] == years_l5[i]]
    landsat5_list.append(sample5) 
    
landsat7_list = []
for i in range(len(years_l7)):
    ls7_x = rasterExtraction(ee.Image(list_7.get(i)))
    fc_dict = ls7_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample7 = ydf[ydf['Year'] == years_l7[i]]
    landsat7_list.append(sample7)  
    
landsat8_list = []
for i in range(len(years_l8)):
    ls8_x = rasterExtraction(ee.Image(list_8.get(i)))
    fc_dict = ls8_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample8 = ydf[ydf['Year'] == years_l8[i]]
    landsat8_list.append(sample8) 

    
l5_extract = pd.concat(landsat5_list)
l7_extract = pd.concat(landsat7_list)
l8_extract = pd.concat(landsat8_list)

landsat_extract = pd.concat([l5_extract,l7_extract,l8_extract])

landsat_extract

,Latitude,Longitude,N,QA_PIXEL,QA_RADSAT,SR_ATMOS_OPACITY,SR_B1,SR_B2,SR_B3,SR_B4,...,ST_QA,ST_TRAD,ST_URAD,Year,flats,liveMass,ndvi,SR_B6,SR_QA_AEROSOL,ST_B10
0,37.167424,-75.941716,3,5440.0,0,225.500000,0.033558,0.046813,0.042247,0.105429,...,226.500000,8575.000000,1152.500000,1999,0.002448,278.613333,0.437003,NaN,NaN,NaN
1,37.180976,-75.940766,4,5440.0,0,226.000000,0.039319,0.063656,0.059476,0.127869,...,227.500000,8381.000000,1150.000000,1999,0.010651,367.160000,0.369888,NaN,NaN,NaN
2,37.181789,-75.942028,7,5440.0,0,227.000000,0.031921,0.044379,0.043801,0.105429,...,254.500000,8603.000000,1149.500000,1999,0.003719,138.354286,0.424350,NaN,NaN,NaN
3,37.346218,-75.901132,2,5440.0,0,262.333333,0.045025,0.066530,0.067731,0.158747,...,316.333333,8584.333333,1903.666667,1999,0.000857,21.200000,0.410342,NaN,NaN,NaN
4,37.395461,-75.876249,3,5440.0,0,235.333333,0.050452,0.077622,0.082425,0.208027,...,392.000000,8824.333333,1894.000000,1999,0.000415,0.000000,0.428295,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296,37.450439,-75.672455,4,21824.0,0,NaN,0.039244,0.041227,0.059120,0.050879,...,375.666667,9023.000000,2194.777778,2017,0.000084,43.440000,0.654672,0.160577,92.666667,300.749834
1297,37.450771,-75.672169,2,21824.0,0,NaN,0.038336,0.038908,0.054519,0.042159,...,377.444444,9002.666667,2194.777778,2017,0.000044,168.480000,0.724454,0.149663,96.000000,300.482849
1298,37.451099,-75.673269,4,21824.0,0,NaN,0.032994,0.033626,0.042385,0.039724,...,354.500000,9048.625000,1967.500000,2017,0.000877,158.240000,0.550368,0.115631,88.500000,301.303980
1279,37.396051,-75.876056,2,21824.0,0,NaN,0.045575,0.049792,0.072800,0.064458,...,430.333333,9390.000000,3823.000000,2018,0.000094,127.520000,0.611018,0.190674,96.000000,309.460088


In [19]:
#Yearly:
years_l5 = range(1999, 2012)
years_l7 = range(2012, 2013)
years_l8 = range(2013, 2019)

# year_extract

landsat5_list = []
for i in range(len(years_l5)):
    ls5_x = rasterExtraction(ee.Image(yearlist_5.get(i)))
    fc_dict = ls5_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample5 = ydf[ydf['Year'] == years_l5[i]]
    landsat5_list.append(sample5) 
    
landsat7_list = []
for i in range(len(years_l7)):
    ls7_x = rasterExtraction(ee.Image(yearlist_7.get(i)))
    fc_dict = ls7_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample7 = ydf[ydf['Year'] == years_l7[i]]
    landsat7_list.append(sample7) 
    
landsat8_list = []
for i in range(len(years_l8)):
    ls8_x = rasterExtraction(ee.Image(yearlist_8.get(i)))
    fc_dict = ls8_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample8 = ydf[ydf['Year'] == years_l8[i]]
    landsat8_list.append(sample8) 

l5_extract = pd.concat(landsat5_list)
l7_extract = pd.concat(landsat7_list)
l8_extract = pd.concat(landsat8_list)

year_extract = pd.concat([l5_extract,l7_extract,l8_extract])

year_extract

,Latitude,Longitude,N,QA_PIXEL,QA_RADSAT,SR_ATMOS_OPACITY,SR_B1,SR_B2,SR_B3,SR_B4,...,ST_QA,ST_TRAD,ST_URAD,Year,flats,liveMass,ndvi,SR_B6,SR_QA_AEROSOL,ST_B10
0,37.167424,-75.941716,3,5440.0,0,184.166667,0.033649,0.046579,0.044278,0.092893,...,384.666667,7689.000000,807.500000,1999,0.007132,278.613333,0.354811,NaN,NaN,NaN
1,37.167670,-75.942334,3,5440.0,0,184.333333,0.034030,0.042623,0.045112,0.092527,...,384.500000,7680.000000,807.500000,1999,0.005820,0.000000,0.345595,NaN,NaN,NaN
2,37.180976,-75.940766,4,5440.0,0,175.600000,0.045113,0.062526,0.062345,0.117278,...,336.200000,7760.800000,917.800000,1999,0.026187,367.160000,0.305280,NaN,NaN,NaN
3,37.181789,-75.942028,7,5440.0,0,175.800000,0.033288,0.043369,0.045707,0.097275,...,349.200000,7938.400000,917.600000,1999,0.008177,138.354286,0.362936,NaN,NaN,NaN
4,37.181867,-75.940537,3,5440.0,0,165.250000,0.036988,0.054375,0.053392,0.107478,...,266.750000,7924.250000,785.250000,1999,0.027201,113.333333,0.336861,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338,37.450439,-75.672455,4,21824.0,0,NaN,0.038885,0.041377,0.056654,0.055728,...,383.666667,8486.200000,1603.200000,2017,0.000502,43.440000,0.565512,0.172796,94.000000,295.623488
1339,37.450771,-75.672169,2,21824.0,0,NaN,0.036619,0.037994,0.050739,0.045885,...,385.200000,8469.733333,1603.133333,2017,0.000411,168.480000,0.621773,0.155188,96.000000,295.421596
1340,37.451099,-75.673269,4,21824.0,0,NaN,0.031978,0.033066,0.040607,0.041843,...,379.785714,8442.928571,1431.000000,2017,0.001605,158.240000,0.486796,0.122104,89.428571,295.589844
1327,37.396051,-75.876056,2,21824.0,0,NaN,0.044970,0.050117,0.067772,0.069170,...,350.333333,8476.166667,2258.666667,2018,0.000817,127.520000,0.497827,0.203237,96.000000,298.606735


In [20]:
#Peak
years_l5 = range(1999, 2012)
years_l7 = range(2012, 2013)
years_l8 = range(2013, 2019)

##PEAK EXTRACT

landsat5_list = []
for i in range(len(years_l5)):
    ls5_x = rasterExtraction(ee.Image(peaklist_5.get(i)))
    fc_dict = ls5_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample5 = ydf[ydf['Year'] == years_l5[i]]
    landsat5_list.append(sample5) 
    
landsat7_list = []
for i in range(len(years_l7)):
    ls7_x = rasterExtraction(ee.Image(peaklist_7.get(i)))
    fc_dict = ls7_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample7 = ydf[ydf['Year'] == years_l7[i]]
    landsat7_list.append(sample7) 
    
landsat8_list = []
for i in range(len(years_l8)):
    ls8_x = rasterExtraction(ee.Image(peaklist_8.get(i)))
    fc_dict = ls8_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample8 = ydf[ydf['Year'] == years_l8[i]]
    landsat8_list.append(sample8) 

l5_extract = pd.concat(landsat5_list)
l7_extract = pd.concat(landsat7_list)
l8_extract = pd.concat(landsat8_list)

peak_extract = pd.concat([l5_extract,l7_extract,l8_extract])

peak_extract

,Latitude,Longitude,N,QA_PIXEL,QA_RADSAT,SR_ATMOS_OPACITY,SR_B1,SR_B2,SR_B3,SR_B4,...,ST_QA,ST_TRAD,ST_URAD,Year,flats,liveMass,ndvi,SR_B6,SR_QA_AEROSOL,ST_B10
0,37.167424,-75.941716,3,5440.0,0,155.0,0.018900,0.039800,0.031055,0.103077,...,227.000000,8326.000000,943.000000,1999,0.001773,278.613333,0.536950,NaN,NaN,NaN
1,37.180976,-75.940766,4,5440.0,0,155.0,0.028525,0.049480,0.046428,0.107560,...,229.000000,8104.000000,942.000000,1999,0.012585,367.160000,0.396997,NaN,NaN,NaN
2,37.181789,-75.942028,7,5440.0,0,155.0,0.023712,0.035015,0.031055,0.103105,...,255.000000,8326.000000,942.000000,1999,0.000965,138.354286,0.537045,NaN,NaN,NaN
3,37.346218,-75.901132,2,5440.0,0,149.0,0.031578,0.049948,0.046785,0.139020,...,260.000000,8215.000000,926.000000,1999,0.001268,21.200000,0.496408,NaN,NaN,NaN
4,37.416357,-75.862222,4,5440.0,0,151.0,0.026820,0.040323,0.039112,0.103490,...,235.000000,8270.000000,918.000000,1999,0.004987,88.400000,0.451447,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296,37.450439,-75.672455,4,21824.0,0,NaN,0.039408,0.040384,0.055991,0.050745,...,359.500000,8852.500000,2024.750000,2017,0.000117,43.440000,0.631984,0.157562,96.0,299.349300
1297,37.450771,-75.672169,2,21824.0,0,NaN,0.037511,0.038061,0.050264,0.045383,...,360.250000,8830.500000,2024.500000,2017,0.000092,168.480000,0.665030,0.147559,96.0,299.096367
1298,37.451099,-75.673269,4,21824.0,0,NaN,0.033269,0.032897,0.038996,0.038171,...,362.500000,8852.250000,2024.750000,2017,0.001032,158.240000,0.529076,0.113452,96.0,299.970525
1261,37.396051,-75.876056,2,21824.0,0,NaN,0.047156,0.051323,0.073144,0.065430,...,500.500000,9302.500000,3680.500000,2018,0.000078,127.520000,0.610335,0.199795,96.0,306.992847


In [21]:
# earlylist_5

In [44]:
##Early EXTRACT
years_l5 = range(1999, 2012)
years_l7 = range(2012, 2013)
years_l8 = range(2013, 2019)

landsat5_list = []
for i in range(len(years_l5)):
    ls5_x = rasterExtraction(ee.Image(earlylist_5.get(i)))
    fc_dict = ls5_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    if features: 
        ydf = pd.DataFrame(features)
        if 'Year' in ydf.columns:
            sample5 = ydf[ydf['Year'] == years_l5[i]]
            landsat5_list.append(sample5)
        else:
            print(f"'Year' column missing for Landsat 5, year {years_l5[i]}")
    else:
        print(f"No features found for Landsat 5, year {years_l5[i]}")
#     ydf = pd.DataFrame(features)
#     sample5 = ydf[ydf['Year'] == years_l5[i]]
#     landsat5_list.append(sample5) 
    
landsat7_list = []
for i in range(len(years_l7)):
    ls7_x = rasterExtraction(ee.Image(earlylist_7.get(i)))
    fc_dict = ls7_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    if features: 
        ydf = pd.DataFrame(features)
        if 'Year' in ydf.columns:
            sample7 = ydf[ydf['Year'] == years_l7[i]]
            landsat7_list.append(sample7)
        else:
            print(f"'Year' column missing for Landsat 7, year {years_l7[i]}")
    else:
        print(f"No features found for Landsat 7, year {years_l7[i]}")
    
landsat8_list = []
for i in range(len(years_l8)):
    ls8_x = rasterExtraction(ee.Image(earlylist_8.get(i)))
    fc_dict = ls8_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    if features: 
        ydf = pd.DataFrame(features)
        if 'Year' in ydf.columns:
            sample8 = ydf[ydf['Year'] == years_l8[i]]
            landsat8_list.append(sample8)
        else:
            print(f"'Year' column missing for Landsat 8, year {years_l8[i]}")
    else:
        print(f"No features found for Landsat 8, year {years_l8[i]}")
    
l5_extract = pd.concat(landsat5_list)
l7_extract = pd.concat(landsat7_list)
l8_extract = pd.concat(landsat8_list)

early_extract = pd.concat([l5_extract,l7_extract,l8_extract])

No features found for Landsat 5, year 2000


In [45]:
early_extract

,Latitude,Longitude,N,QA_PIXEL,QA_RADSAT,SR_ATMOS_OPACITY,SR_B1,SR_B2,SR_B3,SR_B4,...,ST_QA,ST_TRAD,ST_URAD,Year,flats,liveMass,ndvi,SR_B6,SR_QA_AEROSOL,ST_B10
0,37.167424,-75.941716,3,5440.0,0,221.000000,0.040478,0.051002,0.050122,0.095662,...,341.333333,7827.333333,658.333333,1999,0.005010,278.613333,0.311192,NaN,NaN,NaN
1,37.167670,-75.942334,3,5440.0,0,221.000000,0.038727,0.048371,0.050122,0.094433,...,343.666667,7827.666667,658.333333,1999,0.004055,0.000000,0.306023,NaN,NaN,NaN
2,37.287699,-75.929487,2,5440.0,0,223.666667,0.041872,0.059087,0.062497,0.149653,...,381.000000,7532.000000,640.666667,1999,0.000966,669.600000,0.411967,NaN,NaN,NaN
3,37.287773,-75.929452,2,5440.0,0,223.666667,0.041872,0.059087,0.062497,0.149653,...,381.000000,7532.000000,640.666667,1999,0.000966,520.000000,0.411967,NaN,NaN,NaN
4,37.395461,-75.876249,3,5440.0,0,217.666667,0.057739,0.086715,0.096221,0.203315,...,392.000000,8639.666667,1702.000000,1999,0.001150,0.000000,0.348743,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1278,37.450439,-75.672455,4,21824.0,0,NaN,0.039282,0.042550,0.061695,0.055095,...,379.000000,9068.833333,2016.833333,2017,0.000159,43.440000,0.627797,0.177364,91.0,300.791420
1279,37.450771,-75.672169,2,21824.0,0,NaN,0.038521,0.039695,0.057427,0.043792,...,381.666667,9046.000000,2016.833333,2017,0.000115,168.480000,0.714902,0.162180,96.0,300.502027
1280,37.451099,-75.673269,4,21824.0,0,NaN,0.032573,0.034647,0.046070,0.044200,...,344.400000,9105.800000,1617.400000,2017,0.000905,158.240000,0.539268,0.127118,84.0,301.175719
1264,37.396051,-75.876056,2,21824.0,0,NaN,0.045204,0.051130,0.071810,0.072525,...,250.000000,9312.000000,2621.500000,2018,0.000572,127.520000,0.481328,0.216240,96.0,307.503841


In [46]:
##Max EXTRACT LANDSAT

landsat5_list = []
for i in range(len(years_l5)):
    ls5_x = rasterExtraction(ee.Image(Maxlist_5.get(i)))
    fc_dict = ls5_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample5 = ydf[ydf['Year'] == years_l5[i]]
    landsat5_list.append(sample5) 
    
landsat7_list = []
for i in range(len(years_l7)):
    ls7_x = rasterExtraction(ee.Image(Maxlist_7.get(i)))
    fc_dict = ls7_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample7 = ydf[ydf['Year'] == years_l7[i]]
    landsat7_list.append(sample7) 
    
landsat8_list = []
for i in range(len(years_l8)):
    ls8_x = rasterExtraction(ee.Image(Maxlist_8.get(i)))
    fc_dict = ls8_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample8 = ydf[ydf['Year'] == years_l8[i]]
    landsat8_list.append(sample8) 

 
    
l5_extract = pd.concat(landsat5_list)
l7_extract = pd.concat(landsat7_list)
l8_extract = pd.concat(landsat8_list)

Max_extract = pd.concat([l5_extract,l7_extract,l8_extract])

In [47]:
##Min EXTRACT LANDSAT

landsat5_list = []
for i in range(len(years_l5)):
    ls5_x = rasterExtraction(ee.Image(Minlist_5.get(i)))
    fc_dict = ls5_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample5 = ydf[ydf['Year'] == years_l5[i]]
    landsat5_list.append(sample5) 
    
landsat7_list = []
for i in range(len(years_l7)):
    ls7_x = rasterExtraction(ee.Image(Minlist_7.get(i)))
    fc_dict = ls7_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample7 = ydf[ydf['Year'] == years_l7[i]]
    landsat7_list.append(sample7) 
    
landsat8_list = []
for i in range(len(years_l8)):
    ls8_x = rasterExtraction(ee.Image(Minlist_8.get(i)))
    fc_dict = ls8_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sample8 = ydf[ydf['Year'] == years_l8[i]]
    landsat8_list.append(sample8) 


    
l5_extract = pd.concat(landsat5_list)
l7_extract = pd.concat(landsat7_list)
l8_extract = pd.concat(landsat8_list)

Min_extract = pd.concat([l5_extract,l7_extract,l8_extract])

In [48]:
#Daymet for loop and elevation
years_dm = range(1999, 2019)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(ee.Image(dm_list.get(i)))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
daymet_extract = pd.concat(daymet_list)

In [49]:
#Daymet peak time
years_dm = range(1999, 2019)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(ee.Image(peakdm_list.get(i)))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
peakdaymet_extract = pd.concat(daymet_list)

peakdaymet_extract

,Latitude,Longitude,N,Year,dayl,liveMass,prcp,srad,swe,tmax,tmin,vp
0,37.167147,-75.940768,2,1999,43748.101562,46.720,4.225978,327.425232,0,25.539457,16.136957,1920.045410
1,37.167148,-75.943672,4,1999,43748.101562,126.720,4.225978,327.425232,0,25.539457,16.136957,1920.045410
2,37.167154,-75.941307,2,1999,43748.101562,164.960,4.225978,327.425232,0,25.539457,16.136957,1920.045410
3,37.167195,-75.944905,5,1999,43748.101562,83.776,4.225978,327.425232,0,25.539457,16.136957,1920.045410
4,37.167409,-75.940586,2,1999,43748.101562,154.000,4.225978,327.425232,0,25.539457,16.136957,1920.045410
...,...,...,...,...,...,...,...,...,...,...,...,...
1389,37.450439,-75.672455,4,2017,43892.031250,43.440,3.452935,309.518250,0,27.160435,18.834240,2291.298340
1390,37.450771,-75.672169,2,2017,43892.031250,168.480,3.452935,309.518250,0,27.160435,18.834240,2291.298340
1391,37.451099,-75.673269,4,2017,43892.031250,158.240,3.452935,309.518250,0,27.160435,18.834240,2291.298340
1392,37.396051,-75.876056,2,2018,43890.570312,127.520,2.425652,318.734344,0,26.871195,17.770327,2094.927734


In [50]:
#Rolling daymet extraction
years_dm = range(1999, 2019)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(ee.Image(rolling_list.get(i)))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
rolling_extract = pd.concat(daymet_list)

rolling_extract

,Latitude,Longitude,N,Year,dayl,liveMass,prcp,srad,swe,tmax,tmin,vp
0,37.167147,-75.940768,2,1999,43208.375000,46.720,3.223556,316.418457,0.167605,19.660549,9.908839,1410.073242
1,37.167148,-75.943672,4,1999,43208.375000,126.720,3.223556,316.418457,0.167605,19.660549,9.908839,1410.073242
2,37.167154,-75.941307,2,1999,43208.375000,164.960,3.223556,316.418457,0.167605,19.660549,9.908839,1410.073242
3,37.167195,-75.944905,5,1999,43208.375000,83.776,3.223556,316.418457,0.167605,19.660549,9.908839,1410.073242
4,37.167409,-75.940586,2,1999,43208.375000,154.000,3.223556,316.418457,0.167605,19.660549,9.908839,1410.073242
...,...,...,...,...,...,...,...,...,...,...,...,...
1389,37.450439,-75.672455,4,2017,43200.109375,43.440,3.223269,310.242615,0.000000,19.783617,10.596922,1447.260376
1390,37.450771,-75.672169,2,2017,43200.109375,168.480,3.223269,310.242615,0.000000,19.783617,10.596922,1447.260376
1391,37.451099,-75.673269,4,2017,43200.109375,158.240,3.223269,310.242615,0.000000,19.783617,10.596922,1447.260376
1392,37.396051,-75.876056,2,2018,43208.449219,127.520,3.099305,315.034424,0.121920,19.934881,10.111042,1407.385986


In [51]:
#MayOct daymet extraction
years_dm = range(1999, 2019)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(ee.Image(dm_MayOct_list.get(i)))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
dm_MayOct_extract = pd.concat(daymet_list)

dm_MayOct_extract

,Latitude,Longitude,N,Year,dayl,liveMass,prcp,srad,swe,tmax,tmin,vp
0,37.167147,-75.940768,2,1999,47542.890625,46.720,4.833370,352.384827,0,26.652010,17.012554,2005.329346
1,37.167148,-75.943672,4,1999,47542.890625,126.720,4.833370,352.384827,0,26.652010,17.012554,2005.329346
2,37.167154,-75.941307,2,1999,47542.890625,164.960,4.833370,352.384827,0,26.652010,17.012554,2005.329346
3,37.167195,-75.944905,5,1999,47542.890625,83.776,4.833370,352.384827,0,26.652010,17.012554,2005.329346
4,37.167409,-75.940586,2,1999,47542.890625,154.000,4.833370,352.384827,0,26.652010,17.012554,2005.329346
...,...,...,...,...,...,...,...,...,...,...,...,...
1389,37.450439,-75.672455,4,2017,47651.925781,43.440,3.666685,333.727509,0,27.539349,19.095652,2272.918213
1390,37.450771,-75.672169,2,2017,47651.925781,168.480,3.666685,333.727509,0,27.539349,19.095652,2272.918213
1391,37.451099,-75.673269,4,2017,47651.925781,158.240,3.666685,333.727509,0,27.539349,19.095652,2272.918213
1392,37.396051,-75.876056,2,2018,47642.425781,127.520,2.601413,348.369904,0,27.979076,18.553316,2191.562256


In [52]:
#Max daymet extraction
years_dm = range(1999, 2019)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(ee.Image(dm_Max_list.get(i)))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
dm_Max_extract = pd.concat(daymet_list)

dm_Max_extract

,Latitude,Longitude,N,Year,dayl,liveMass,prcp,srad,swe,tmax,tmin,vp
0,37.167147,-75.940768,2,1999,52410.191406,46.720,47.299999,551.260010,0,33.630001,25.459999,3254.989990
1,37.167148,-75.943672,4,1999,52410.191406,126.720,47.299999,551.260010,0,33.630001,25.459999,3254.989990
2,37.167154,-75.941307,2,1999,52410.191406,164.960,47.299999,551.260010,0,33.630001,25.459999,3254.989990
3,37.167195,-75.944905,5,1999,52410.191406,83.776,47.299999,551.260010,0,33.630001,25.459999,3254.989990
4,37.167409,-75.940586,2,1999,52410.191406,154.000,47.299999,551.260010,0,33.630001,25.459999,3254.989990
...,...,...,...,...,...,...,...,...,...,...,...,...
1389,37.450439,-75.672455,4,2017,52510.050781,43.440,63.610001,561.489990,0,34.919998,24.889999,3146.830078
1390,37.450771,-75.672169,2,2017,52510.050781,168.480,63.610001,561.489990,0,34.919998,24.889999,3146.830078
1391,37.451099,-75.673269,4,2017,52510.050781,158.240,63.610001,561.489990,0,34.919998,24.889999,3146.830078
1392,37.396051,-75.876056,2,2018,52490.011719,127.520,68.919998,528.619995,0,35.889999,26.719999,3507.010010


In [53]:
#Min daymet extraction
years_dm = range(1999, 2019)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(ee.Image(dm_Min_list.get(i)))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
dm_Min_extract = pd.concat(daymet_list)

dm_Min_extract

,Latitude,Longitude,N,Year,dayl,liveMass,prcp,srad,swe,tmax,tmin,vp
0,37.167147,-75.940768,2,1999,52410.191406,46.720,47.299999,551.260010,0,33.630001,25.459999,3254.989990
1,37.167148,-75.943672,4,1999,52410.191406,126.720,47.299999,551.260010,0,33.630001,25.459999,3254.989990
2,37.167154,-75.941307,2,1999,52410.191406,164.960,47.299999,551.260010,0,33.630001,25.459999,3254.989990
3,37.167195,-75.944905,5,1999,52410.191406,83.776,47.299999,551.260010,0,33.630001,25.459999,3254.989990
4,37.167409,-75.940586,2,1999,52410.191406,154.000,47.299999,551.260010,0,33.630001,25.459999,3254.989990
...,...,...,...,...,...,...,...,...,...,...,...,...
1389,37.450439,-75.672455,4,2017,52510.050781,43.440,63.610001,561.489990,0,34.919998,24.889999,3146.830078
1390,37.450771,-75.672169,2,2017,52510.050781,168.480,63.610001,561.489990,0,34.919998,24.889999,3146.830078
1391,37.451099,-75.673269,4,2017,52510.050781,158.240,63.610001,561.489990,0,34.919998,24.889999,3146.830078
1392,37.396051,-75.876056,2,2018,52490.011719,127.520,68.919998,528.619995,0,35.889999,26.719999,3507.010010


In [54]:
#Early daymet extraction
years_dm = range(1999, 2019)

daymet_list = []
for i in range(len(years_dm)):
    dm_x = rasterExtraction(ee.Image(early_list.get(i)))
    fc_dict = dm_x.getInfo()
    features = [feature['properties'] for feature in fc_dict['features']]
    ydf = pd.DataFrame(features)
    sampledm = ydf[ydf['Year'] == years_dm[i]]
    daymet_list.append(sampledm) 
    
earlydaymet_extract = pd.concat(daymet_list)

earlydaymet_extract

,Latitude,Longitude,N,Year,dayl,liveMass,prcp,srad,swe,tmax,tmin,vp
0,37.167147,-75.940768,2,1999,48660.390625,46.720,4.800000,363.113464,0.0,23.731699,13.499608,1662.920288
1,37.167148,-75.943672,4,1999,48660.390625,126.720,4.800000,363.113464,0.0,23.731699,13.499608,1662.920288
2,37.167154,-75.941307,2,1999,48660.390625,164.960,4.800000,363.113464,0.0,23.731699,13.499608,1662.920288
3,37.167195,-75.944905,5,1999,48660.390625,83.776,4.800000,363.113464,0.0,23.731699,13.499608,1662.920288
4,37.167409,-75.940586,2,1999,48660.390625,154.000,4.800000,363.113464,0.0,23.731699,13.499608,1662.920288
...,...,...,...,...,...,...,...,...,...,...,...,...
1389,37.450439,-75.672455,4,2017,48653.960938,43.440,3.202288,360.370270,0.0,22.410131,13.490392,1709.885010
1390,37.450771,-75.672169,2,2017,48653.960938,168.480,3.202288,360.370270,0.0,22.410131,13.490392,1709.885010
1391,37.451099,-75.673269,4,2017,48653.960938,158.240,3.202288,360.370270,0.0,22.410131,13.490392,1709.885010
1392,37.396051,-75.876056,2,2018,48642.332031,127.520,2.932680,373.041107,0.0,24.258888,14.336013,1798.497559


In [56]:
#Final steps - extra rows in merge?

# landsat_extract = MayOct_extract ##NOT sure if this is the right way to do this - 
#where are all these dfs stored? (in streamlined folder in biomass datasets/sapelo)

#rolling_extract, earlydaymet_extract, early_extract (Landsat)

#MayOct and elevation
# landsat_extract = MayOct_extract
dfx = pd.merge(landsat_extract, dem_vals, how = 'left')

#dfx and AugOct
peak_extract.rename(
    columns={
        'SR_B1':'SR_B1_peak','SR_B2':'SR_B2_peak','SR_B3':'SR_B3_peak','SR_B4':'SR_B4_peak', 'SR_B5':'SR_B5_peak',
        'SR_B6':'SR_B6_peak', 'SR_B7':'SR_B7_peak', 'flats':'flats_peak'
    }, inplace=True
)

dfx1 = pd.merge(dfx, peak_extract, on = ['liveMass', 'Latitude', 'Longitude', 'Year'], how='outer',
                suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

#dfx1 and year_extract

year_extract.rename(
    columns={
        'SR_B1':'SR_B1_year','SR_B2':'SR_B2_year','SR_B3':'SR_B3_year','SR_B4':'SR_B4_year', 'SR_B5':'SR_B5_year',
        'SR_B6':'SR_B6_year', 'SR_B7':'SR_B7_year', 'flats':'flats_year'
    }, inplace=True
)

dfx2 = pd.merge(dfx1, year_extract, on = ['liveMass', 'Latitude', 'Longitude', 'Year'], how='outer',
                suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')



#dfx2 and early_extract
early_extract.rename(
    columns={
        'SR_B1':'SR_B1_MarMay','SR_B2':'SR_B2_MarMay','SR_B3':'SR_B3_MarMay','SR_B4':'SR_B4_MarMay', 'SR_B5':'SR_B5_MarMay',
        'SR_B6':'SR_B6_MarMay', 'SR_B7':'SR_B7_MarMay', 'flats':'flats_MarMay'
    }, inplace=True
)

dfx2_a = pd.merge(dfx2, early_extract, on = ['liveMass', 'Latitude', 'Longitude', 'Year'], how='outer',
                suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

#dfx2_a and max
Max_extract.rename(
    columns={
        'SR_B1':'SR_B1_Max','SR_B2':'SR_B2_Max','SR_B3':'SR_B3_Max','SR_B4':'SR_B4_Max', 'SR_B5':'SR_B5_Max',
        'SR_B6':'SR_B6_Max', 'SR_B7':'SR_B7_Max', 'flats':'flats_Max'
    }, inplace=True
)

dfx2_b = pd.merge(dfx2_a, Max_extract, on = ['liveMass', 'Latitude', 'Longitude', 'Year'], how='outer',
                suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

#dfx2_b and min
Min_extract.rename(
    columns={
        'SR_B1':'SR_B1_Min','SR_B2':'SR_B2_Min','SR_B3':'SR_B3_Min','SR_B4':'SR_B4_Min', 'SR_B5':'SR_B5_Min',
        'SR_B6':'SR_B6_Min', 'SR_B7':'SR_B7_Min', 'flats':'flats_Min'
    }, inplace=True
)

dfx2_c = pd.merge(dfx2_b, Min_extract, on = ['liveMass', 'Latitude', 'Longitude', 'Year'], how='outer',
                suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')


#dfx2 and daymet mayoct OR YEAR
dfx3 = pd.merge(dfx2_c, daymet_extract, on = ['liveMass', 'Latitude', 'Longitude', 'Year'], how='right')

#dfx3 and daymet rolling average
rolling_extract.rename(
    columns={
        'swe':'swe_rolling','tmax':'tmax_rolling','tmin':'tmin_rolling','srad':'srad_rolling', 'vp':'vp_rolling',
        'prcp':'prcp_rolling', 'dayl':'dayl_rolling'
    }, inplace=True
)

dfx3_a = pd.merge(dfx3, rolling_extract, on = ['liveMass', 'Latitude', 'Longitude', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

#dfx3_a and daymet early (marMay)
earlydaymet_extract.rename(
    columns={
        'swe':'swe_daymMarMay','tmax':'tmax_daymMarMay','tmin':'tmin_daymMarMay','srad':'srad_daymMarMay', 'vp':'vp_daymMarMay',
        'prcp':'prcp_daymMarMay', 'dayl':'dayl_daymMarMay'
    }, inplace=True
)

dfx3_b = pd.merge(dfx3_a, earlydaymet_extract, on = ['liveMass', 'Latitude', 'Longitude', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

#dfx3_b and daymet growing (MayOct)
dm_MayOct_extract.rename(
    columns={
        'swe':'swe_daymMayOct','tmax':'tmax_daymMayOct','tmin':'tmin_daymMayOct','srad':'srad_daymMayOct', 'vp':'vp_daymMayOct',
        'prcp':'prcp_daymMayOct', 'dayl':'dayl_daymMayOct'
    }, inplace=True
)

dfx3_c = pd.merge(dfx3_b, dm_MayOct_extract, on = ['liveMass', 'Latitude', 'Longitude', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')


#dfx3_c and daymet max (MayOct)
dm_Max_extract.rename(
    columns={
        'swe':'swe_Max','tmax':'tmax_Max','tmin':'tmin_Max','srad':'srad_Max', 'vp':'vp_Max',
        'prcp':'prcp_Max', 'dayl':'dayl_Max'
    }, inplace=True
)

dfx3_d = pd.merge(dfx3_c, dm_Max_extract, on = ['liveMass', 'Latitude', 'Longitude', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')


#dfx3_d and daymet min (MayOct)
dm_Min_extract.rename(
    columns={
        'swe':'swe_Min','tmax':'tmax_Min','tmin':'tmin_Min','srad':'srad_Min', 'vp':'vp_Min',
        'prcp':'prcp_Min', 'dayl':'dayl_Min'
    }, inplace=True
)

dfx3_e = pd.merge(dfx3_d, dm_Min_extract, on = ['liveMass', 'Latitude', 'Longitude', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

#dfx3_e and daymet peak
peakdaymet_extract.rename(
    columns={
        'swe':'swe_peak','tmax':'tmax_peak','tmin':'tmin_peak','srad':'srad_peak', 'vp':'vp_peak',
        'prcp':'prcp_peak', 'dayl':'dayl_peak'
    }, inplace=True
)

df = pd.merge(dfx3_e, peakdaymet_extract, on = ['liveMass', 'Latitude', 'Longitude', 'Year'], how='right',
              suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')


display(df)

,Latitude,Longitude,N_x,QA_PIXEL,QA_RADSAT,SR_ATMOS_OPACITY,SR_B1,SR_B2,SR_B3,SR_B4,...,tmax_Min,tmin_Min,vp_Min,dayl_peak,prcp_peak,srad_peak,swe_peak,tmax_peak,tmin_peak,vp_peak
0,37.167147,-75.940768,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,33.630001,25.459999,3254.989990,43748.101562,4.225978,327.425232,0,25.539457,16.136957,1920.045410
1,37.167148,-75.943672,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,33.630001,25.459999,3254.989990,43748.101562,4.225978,327.425232,0,25.539457,16.136957,1920.045410
2,37.167154,-75.941307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,33.630001,25.459999,3254.989990,43748.101562,4.225978,327.425232,0,25.539457,16.136957,1920.045410
3,37.167195,-75.944905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,33.630001,25.459999,3254.989990,43748.101562,4.225978,327.425232,0,25.539457,16.136957,1920.045410
4,37.167409,-75.940586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,33.630001,25.459999,3254.989990,43748.101562,4.225978,327.425232,0,25.539457,16.136957,1920.045410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389,37.450439,-75.672455,4.0,21824.0,0.0,NaN,0.039244,0.041227,0.059120,0.050879,...,34.919998,24.889999,3146.830078,43892.031250,3.452935,309.518250,0,27.160435,18.834240,2291.298340
1390,37.450771,-75.672169,2.0,21824.0,0.0,NaN,0.038336,0.038908,0.054519,0.042159,...,34.919998,24.889999,3146.830078,43892.031250,3.452935,309.518250,0,27.160435,18.834240,2291.298340
1391,37.451099,-75.673269,4.0,21824.0,0.0,NaN,0.032994,0.033626,0.042385,0.039724,...,34.919998,24.889999,3146.830078,43892.031250,3.452935,309.518250,0,27.160435,18.834240,2291.298340
1392,37.396051,-75.876056,2.0,21824.0,0.0,NaN,0.045575,0.049792,0.072800,0.064458,...,35.889999,26.719999,3507.010010,43890.570312,2.425652,318.734344,0,26.871195,17.770327,2094.927734


In [ ]:
# #Final steps - extra rows in merge?

# #MayOct and elevation
# dfx = pd.merge(landsat_extract, dem_vals, how = 'left')

# #dfx and AugOct
# peak_extract.rename(
#     columns={
#         'SR_B1':'SR_B1_peak','SR_B2':'SR_B2_peak','SR_B3':'SR_B3_peak','SR_B4':'SR_B4_peak', 'SR_B5':'SR_B5_peak',
#         'SR_B6':'SR_B6_peak', 'SR_B7':'SR_B7_peak', 'flats':'flats_peak'
#     }, inplace=True
# )

# dfx1 = pd.merge(dfx, peak_extract, on = ['liveMass', 'Transect', 'locationID', 'Year'], how='outer',
#                 suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

# #dfx1 and year_extract

# year_extract.rename(
#     columns={
#         'SR_B1':'SR_B1_year','SR_B2':'SR_B2_year','SR_B3':'SR_B3_year','SR_B4':'SR_B4_year', 'SR_B5':'SR_B5_year',
#         'SR_B6':'SR_B6_year', 'SR_B7':'SR_B7_year', 'flats':'flats_year'
#     }, inplace=True
# )

# dfx2 = pd.merge(dfx1, year_extract, on = ['liveMass', 'Transect', 'locationID', 'Year'], how='outer',
#                 suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')


# #dfx2 and daymet mayoct
# dfx3 = pd.merge(dfx2, daymet_extract, on = ['liveMass', 'Transect', 'locationID', 'Year'], how='right')

# #dfx3 and daymet peak
# peakdaymet_extract.rename(
#     columns={
#         'swe':'swe_peak','tmax':'tmax_peak','tmin':'tmin_peak','srad':'srad_peak', 'vp':'vp_peak',
#         'prcp':'prcp_peak', 'dayl':'dayl_peak'
#     }, inplace=True
# )

# df = pd.merge(dfx3, peakdaymet_extract, on = ['liveMass', 'Transect', 'locationID', 'Year'], how='right',
#               suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)')

# display(df)

In [57]:
##Bands and indices
df['Sensor'] = np.where(df['Year']<2013, 'Landsat 5', 'Landsat 8') ##make sure no other sensors are being used

df.loc[df['Year'] == 2012, 'Sensor'] = 'Landsat 7'

df['ndvi'] = np.where(df['Sensor'] == 'Landsat 8', (df['SR_B5']-df['SR_B4'])/(df['SR_B5']+df['SR_B4']), \
                      (df['SR_B4']-df['SR_B3'])/(df['SR_B4']+df['SR_B3'])) 
##ndvi conditional based on whether sensor is Landsat-5 or 8

df['Blue_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B2'], df['SR_B1'])
df['Green_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B3'], df['SR_B2'])
df['Red_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B4'], df['SR_B3'])
df['NIR_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B5'], df['SR_B4'])
df['SWIR1_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B6'], df['SR_B5'])
df['SWIR2_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B7'], df['SR_B7'])

##Variables from Byrd et al. 2018 (make sure calculations are accurate):
df['savi'] = ((df['NIR_band']-df['Red_band'])*1.5)/(df['NIR_band']+df['Red_band']+0.5)
df['wdrvi5'] = (0.5*df['NIR_band']-df['Red_band'])/(0.5*df['NIR_band']+df['Red_band'])
df['nd_r_g'] = (df['Red_band']-df['Green_band'])/(df['Red_band']+df['Green_band'])
df['nd_g_b'] = (df['Green_band']-df['Blue_band'])/(df['Green_band']+df['Blue_band'])
df['nd_swir2_nir'] = (df['SWIR2_band']-df['NIR_band'])/(df['SWIR2_band']+df['NIR_band'])
df['nd_swir2_r'] = (df['SWIR2_band']-df['Red_band'])/(df['SWIR2_band']+df['Red_band'])

display(df)

##EXPORT
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'df_mayoct.csv')
# df.to_csv(out_csv, index = False)

,Latitude,Longitude,N_x,QA_PIXEL,QA_RADSAT,SR_ATMOS_OPACITY,SR_B1,SR_B2,SR_B3,SR_B4,...,Red_band,NIR_band,SWIR1_band,SWIR2_band,savi,wdrvi5,nd_r_g,nd_g_b,nd_swir2_nir,nd_swir2_r
0,37.167147,-75.940768,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,37.167148,-75.943672,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,37.167154,-75.941307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,37.167195,-75.944905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,37.167409,-75.940586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389,37.450439,-75.672455,4.0,21824.0,0.0,NaN,0.039244,0.041227,0.059120,0.050879,...,0.050879,0.245806,0.160577,0.080039,0.367007,0.414446,-0.074917,0.178314,-0.508731,0.222728
1390,37.450771,-75.672169,2.0,21824.0,0.0,NaN,0.038336,0.038908,0.054519,0.042159,...,0.042159,0.272655,0.149663,0.068146,0.424323,0.527595,-0.127845,0.167092,-0.600081,0.235596
1391,37.451099,-75.673269,4.0,21824.0,0.0,NaN,0.032994,0.033626,0.042385,0.039724,...,0.039724,0.137827,0.115631,0.062890,0.217185,0.268683,-0.032403,0.115230,-0.373349,0.225752
1392,37.396051,-75.876056,2.0,21824.0,0.0,NaN,0.045575,0.049792,0.072800,0.064458,...,0.064458,0.268976,0.190674,0.095992,0.368087,0.352003,-0.060773,0.187683,-0.473971,0.196531


In [58]:
df['ndvi_year'] = np.where(df['Sensor'] == 'Landsat 8', (df['SR_B5_year']-df['SR_B4_year'])/(df['SR_B5_year']+df['SR_B4_year']), \
                      (df['SR_B4_year']-df['SR_B3_year'])/(df['SR_B4_year']+df['SR_B3_year'])) 
##ndvi conditional based on whether sensor is Landsat-5 or 8

df['Blue_band_year'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B2_year'], df['SR_B1_year'])
df['Green_band_year'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B3_year'], df['SR_B2_year'])
df['Red_band_year'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B4_year'], df['SR_B3_year'])
df['NIR_band_year'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B5_year'], df['SR_B4_year'])
df['SWIR1_band_year'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B6_year'], df['SR_B5_year'])
df['SWIR2_band_year'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B7_year'], df['SR_B7_year'])

##Variables from Byrd et al. 2018 (make sure calculations are accurate):
df['savi_year'] = ((df['NIR_band_year']-df['Red_band_year'])*1.5)/(df['NIR_band_year']+df['Red_band_year']+0.5)
df['wdrvi5_year'] = (0.5*df['NIR_band_year']-df['Red_band_year'])/(0.5*df['NIR_band_year']+df['Red_band_year'])
df['nd_r_g_year'] = (df['Red_band_year']-df['Green_band_year'])/(df['Red_band_year']+df['Green_band_year'])
df['nd_g_b_year'] = (df['Green_band_year']-df['Blue_band_year'])/(df['Green_band_year']+df['Blue_band_year'])
df['nd_swir2_nir_year'] = (df['SWIR2_band_year']-df['NIR_band_year'])/(df['SWIR2_band_year']+df['NIR_band_year'])
df['nd_swir2_r_year'] = (df['SWIR2_band_year']-df['Red_band_year'])/(df['SWIR2_band_year']+df['Red_band_year'])

In [59]:
df['ndvi_peak'] = np.where(df['Sensor'] == 'Landsat 8', (df['SR_B5_peak']-df['SR_B4_peak'])/(df['SR_B5_peak']+df['SR_B4_peak']), \
                      (df['SR_B4_peak']-df['SR_B3_peak'])/(df['SR_B4_peak']+df['SR_B3_peak'])) 
##ndvi conditional based on whether sensor is Landsat-5 or 8

df['Blue_band_peak'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B2_peak'], df['SR_B1_peak'])
df['Green_band_peak'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B3_peak'], df['SR_B2_peak'])
df['Red_band_peak'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B4_peak'], df['SR_B3_peak'])
df['NIR_band_peak'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B5_peak'], df['SR_B4_peak'])
df['SWIR1_band_peak'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B6_peak'], df['SR_B5_peak'])
df['SWIR2_band_peak'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B7_peak'], df['SR_B7_peak'])

##Variables from Byrd et al. 2018 (make sure calculations are accurate):
df['savi_peak'] = ((df['NIR_band_peak']-df['Red_band_peak'])*1.5)/(df['NIR_band_peak']+df['Red_band_peak']+0.5)
df['wdrvi5_peak'] = (0.5*df['NIR_band_peak']-df['Red_band_peak'])/(0.5*df['NIR_band_peak']+df['Red_band_peak'])
df['nd_r_g_peak'] = (df['Red_band_peak']-df['Green_band_peak'])/(df['Red_band_peak']+df['Green_band_peak'])
df['nd_g_b_peak'] = (df['Green_band_peak']-df['Blue_band_peak'])/(df['Green_band_peak']+df['Blue_band_peak'])
df['nd_swir2_nir_peak'] = (df['SWIR2_band_peak']-df['NIR_band_peak'])/(df['SWIR2_band_peak']+df['NIR_band_peak'])
df['nd_swir2_r_peak'] = (df['SWIR2_band_peak']-df['Red_band_peak'])/(df['SWIR2_band_peak']+df['Red_band_peak'])

In [63]:
df['ndvi_MarMay'] = np.where(df['Sensor'] == 'Landsat 8', (df['SR_B5_MarMay']-df['SR_B4_MarMay'])/(df['SR_B5_MarMay']+df['SR_B4_MarMay']), \
                      (df['SR_B4_MarMay']-df['SR_B3_MarMay'])/(df['SR_B4_MarMay']+df['SR_B3_MarMay'])) 
##ndvi conditional based on whether sensor is Landsat-5 or 8

df['Blue_band_MarMay'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B2_MarMay'], df['SR_B1_MarMay'])
df['Green_band_MarMay'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B3_MarMay'], df['SR_B2_MarMay'])
df['Red_band_MarMay'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B4_MarMay'], df['SR_B3_MarMay'])
df['NIR_band_MarMay'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B5_MarMay'], df['SR_B4_MarMay'])
df['SWIR1_band_MarMay'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B6_MarMay'], df['SR_B5_MarMay'])
df['SWIR2_band_MarMay'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B7_MarMay'], df['SR_B7_MarMay'])

##Variables from Byrd et al. 2018 (make sure calculations are accurate):
df['savi_MarMay'] = ((df['NIR_band_MarMay']-df['Red_band_MarMay'])*1.5)/(df['NIR_band_MarMay']+df['Red_band_MarMay']+0.5)
df['wdrvi5_MarMay'] = (0.5*df['NIR_band_MarMay']-df['Red_band_MarMay'])/(0.5*df['NIR_band_MarMay']+df['Red_band_MarMay'])
df['nd_r_g_MarMay'] = (df['Red_band_MarMay']-df['Green_band_MarMay'])/(df['Red_band_MarMay']+df['Green_band_MarMay'])
df['nd_g_b_MarMay'] = (df['Green_band_MarMay']-df['Blue_band_MarMay'])/(df['Green_band_MarMay']+df['Blue_band_MarMay'])
df['nd_swir2_nir_MarMay'] = (df['SWIR2_band_MarMay']-df['NIR_band_MarMay'])/(df['SWIR2_band_MarMay']+df['NIR_band_MarMay'])
df['nd_swir2_r_MarMay'] = (df['SWIR2_band_MarMay']-df['Red_band_MarMay'])/(df['SWIR2_band_MarMay']+df['Red_band_MarMay'])

In [64]:
##Bands and indices
df['Sensor'] = np.where(df['Year']<2013, 'Landsat 5', 'Landsat 8') ##make sure no other sensors are being used

df.loc[df['Year'] == 2012, 'Sensor'] = 'Landsat 7'

df['ndvi'] = np.where(df['Sensor'] == 'Landsat 8', (df['SR_B5']-df['SR_B4'])/(df['SR_B5']+df['SR_B4']), \
                      (df['SR_B4']-df['SR_B3'])/(df['SR_B4']+df['SR_B3'])) 
##ndvi conditional based on whether sensor is Landsat-5 or 8

df['Blue_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B2'], df['SR_B1'])
df['Green_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B3'], df['SR_B2'])
df['Red_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B4'], df['SR_B3'])
df['NIR_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B5'], df['SR_B4'])
df['SWIR1_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B6'], df['SR_B5'])
df['SWIR2_band'] = np.where(df['Sensor'] == 'Landsat 8', df['SR_B7'], df['SR_B7'])

##Variables from Byrd et al. 2018 (make sure calculations are accurate):
df['savi'] = ((df['NIR_band']-df['Red_band'])*1.5)/(df['NIR_band']+df['Red_band']+0.5)
df['wdrvi5'] = (0.5*df['NIR_band']-df['Red_band'])/(0.5*df['NIR_band']+df['Red_band'])
df['nd_r_g'] = (df['Red_band']-df['Green_band'])/(df['Red_band']+df['Green_band'])
df['nd_g_b'] = (df['Green_band']-df['Blue_band'])/(df['Green_band']+df['Blue_band'])
df['nd_swir2_nir'] = (df['SWIR2_band']-df['NIR_band'])/(df['SWIR2_band']+df['NIR_band'])
df['nd_swir2_r'] = (df['SWIR2_band']-df['Red_band'])/(df['SWIR2_band']+df['Red_band'])

display(df)

##EXPORT
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'va_min.csv')
# df.to_csv(out_csv, index = False)

,Latitude,Longitude,N_x,QA_PIXEL,QA_RADSAT,SR_ATMOS_OPACITY,SR_B1,SR_B2,SR_B3,SR_B4,...,Red_band_MarMay,NIR_band_MarMay,SWIR1_band_MarMay,SWIR2_band_MarMay,savi_MarMay,wdrvi5_MarMay,nd_r_g_MarMay,nd_g_b_MarMay,nd_swir2_nir_MarMay,nd_swir2_r_MarMay
5,37.167424,-75.941716,3.0,5440.0,0.0,225.500000,0.033558,0.046813,0.042247,0.105429,...,0.050122,0.095662,0.122877,0.078318,0.105779,-0.023387,-0.008702,0.115034,-0.099686,0.219532
30,37.180976,-75.940766,4.0,5440.0,0.0,226.000000,0.039319,0.063656,0.059476,0.127869,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,37.181789,-75.942028,7.0,5440.0,0.0,227.000000,0.031921,0.044379,0.043801,0.105429,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,37.346218,-75.901132,2.0,5440.0,0.0,262.333333,0.045025,0.066530,0.067731,0.158747,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,37.416357,-75.862222,4.0,5440.0,0.0,203.500000,0.036761,0.049934,0.048531,0.109471,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389,37.450439,-75.672455,4.0,21824.0,0.0,NaN,0.039244,0.041227,0.059120,0.050879,...,0.055095,0.246971,0.177364,0.092861,0.358842,0.382971,-0.056512,0.183651,-0.453488,0.255256
1390,37.450771,-75.672169,2.0,21824.0,0.0,NaN,0.038336,0.038908,0.054519,0.042159,...,0.043792,0.283349,0.162180,0.077315,0.434431,0.527763,-0.134711,0.182584,-0.571265,0.276801
1391,37.451099,-75.673269,4.0,21824.0,0.0,NaN,0.032994,0.033626,0.042385,0.039724,...,0.044200,0.147737,0.127118,0.072739,0.224451,0.251284,-0.020716,0.141526,-0.340162,0.244054
1392,37.396051,-75.876056,2.0,21824.0,0.0,NaN,0.045575,0.049792,0.072800,0.064458,...,0.072525,0.215786,0.216240,0.134716,0.272598,0.196034,0.004954,0.168212,-0.231296,0.300091


In [65]:
##Next step: ML
df = df[df['Red_band'].notna()]
df = df[df['SR_B4_peak'].notna()]
df = df[df['SR_B4_year'].notna()]
df


,Latitude,Longitude,N_x,QA_PIXEL,QA_RADSAT,SR_ATMOS_OPACITY,SR_B1,SR_B2,SR_B3,SR_B4,...,Red_band_MarMay,NIR_band_MarMay,SWIR1_band_MarMay,SWIR2_band_MarMay,savi_MarMay,wdrvi5_MarMay,nd_r_g_MarMay,nd_g_b_MarMay,nd_swir2_nir_MarMay,nd_swir2_r_MarMay
5,37.167424,-75.941716,3.0,5440.0,0.0,225.500000,0.033558,0.046813,0.042247,0.105429,...,0.050122,0.095662,0.122877,0.078318,0.105779,-0.023387,-0.008702,0.115034,-0.099686,0.219532
30,37.180976,-75.940766,4.0,5440.0,0.0,226.000000,0.039319,0.063656,0.059476,0.127869,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,37.181789,-75.942028,7.0,5440.0,0.0,227.000000,0.031921,0.044379,0.043801,0.105429,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,37.346218,-75.901132,2.0,5440.0,0.0,262.333333,0.045025,0.066530,0.067731,0.158747,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,37.416357,-75.862222,4.0,5440.0,0.0,203.500000,0.036761,0.049934,0.048531,0.109471,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389,37.450439,-75.672455,4.0,21824.0,0.0,NaN,0.039244,0.041227,0.059120,0.050879,...,0.055095,0.246971,0.177364,0.092861,0.358842,0.382971,-0.056512,0.183651,-0.453488,0.255256
1390,37.450771,-75.672169,2.0,21824.0,0.0,NaN,0.038336,0.038908,0.054519,0.042159,...,0.043792,0.283349,0.162180,0.077315,0.434431,0.527763,-0.134711,0.182584,-0.571265,0.276801
1391,37.451099,-75.673269,4.0,21824.0,0.0,NaN,0.032994,0.033626,0.042385,0.039724,...,0.044200,0.147737,0.127118,0.072739,0.224451,0.251284,-0.020716,0.141526,-0.340162,0.244054
1392,37.396051,-75.876056,2.0,21824.0,0.0,NaN,0.045575,0.049792,0.072800,0.064458,...,0.072525,0.215786,0.216240,0.134716,0.272598,0.196034,0.004954,0.168212,-0.231296,0.300091


In [66]:
#EXPORT
out_dir = os.path.expanduser('~/Downloads')
out_csv = os.path.join(out_dir, 'va_testzzz.csv')
# df.to_csv(out_csv, index = False)

In [ ]:
#:)